In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Ankylosing_Spondylitis/GSE25101'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling in whole blood in ankylosing spondylitis patients and controls"
!Series_summary	"Introduction: A number of genetic-association studies have identified genes contributing to AS susceptibility but such approaches provide little information as to the gene activity changes occurring during the disease process. Transcriptional profiling generates a “snapshot” of the sampled cells activity and thus can provide insights into the molecular processes driving the disease process. We undertook a whole-genome microarray approach to identify candidate genes associated with AS and validated these gene-expression changes in a larger sample cohort.  Methods: 18 active AS patients, classified according to the New York criteria. and 18 gender-and age-matched controls were profiled using Illumina HT-12 Whole-Genome Expression BeadChips which carry cDNAs for 48000 genes and transcripts. Class comparison analysis identified a number of differentia

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Based on the background information, this dataset contains gene expression data
is_gene_available = True

# Sample Characteristics Dictionary:
sample_characteristics = {
    0: ['tissue: Whole blood'], 
    1: ['cell type: PBMC'], 
    2: ['disease status: Ankylosing spondylitis patient', 'disease status: Normal control']
}

# Determine the keys for Ankylosing_Spondylitis, age, and gender
if len(set([d.split(': ')[1] for d in sample_characteristics[2]])) > 1:
    trait_row = 2

# No information about age and gender in the sample_characteristics dictionary
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    """
    Convert Ankylosing_Spondylitis values to binary.
    """
    actual_value = value.split(': ')[1]
    if actual_value == 'Ankylosing spondylitis patient':
        return 1
    elif actual_value == 'Normal control':
        return 0
    else:
        return None

convert_age = lambda value: None  # As age data is not available
convert_gender = lambda value: None  # As gender data is not available

# Save cohort information
save_cohort_info('GSE25101', './preprocessed/Ankylosing_Spondylitis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Ankylosing_Spondylitis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Ankylosing_Spondylitis/trait_data/GSE25101.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM616668': [1], 'GSM616669': [1], 'GSM616670': [1], 'GSM616671': [1], 'GSM616672': [1], 'GSM616673': [1], 'GSM616674': [1], 'GSM616675': [1], 'GSM616676': [1], 'GSM616677': [1], 'GSM616678': [1], 'GSM616679': [1], 'GSM616680': [1], 'GSM616681': [1], 'GSM616682': [1], 'GSM616683': [1], 'GSM616684': [0], 'GSM616685': [0], 'GSM616686': [0], 'GSM616687': [0], 'GSM616688': [0], 'GSM616689': [0], 'GSM616690': [0], 'GSM616691': [0], 'GSM616692': [0], 'GSM616693': [0], 'GSM616694': [0], 'GSM616695': [0], 'GSM616696': [0], 'GSM616697': [0], 'GSM616698': [0], 'GSM616699': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228',
       'ILMN_1651229', 'ILMN_1651232', 'ILMN_1651237', 'ILMN_1651254',
       'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278', 'ILMN_1651282',
       'ILMN_1651286', 'ILMN_1651296', 'ILMN_1651315', 'ILMN_1651316',
       'ILMN_1651328', 'ILMN_1651336', 'ILMN_1651341', 'ILMN_1651346'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:

# 1. Identify the keys in the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'ILMN_Gene'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_data = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_data)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Ankylosing_Spondylitis/gene_data/GSE25101.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Ankylosing_Spondylitis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE25101', './preprocessed/Ankylosing_Spondylitis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Ankylosing_Spondylitis/GSE25101.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Ankylosing_Spondylitis', the least common label is '1.0' with 16 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Ankylosing_Spondylitis' in this dataset is fine.

